In [35]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
import imageio
import os
import random

### Data load & split into train and test set

In [36]:
## Image loader
def load_image(img_dir):
    img_list = os.listdir(img_dir) # image file list load
    img_data = []
    for f_name in img_list:
        img_data.append(imageio.imread(img_dir+f_name)/255)
    return np.asarray(img_data)
        

In [37]:
lever_img_dir = './Door_imgs/lever/resize/'
round_img_dir = './Door_imgs/round/resize/'
lever_imgs = load_image(lever_img_dir)
round_imgs = load_image(round_img_dir)

In [38]:
print('Number of lever images: ', lever_imgs.shape)
print('Number of round images: ', round_imgs.shape)

Number of lever images:  (2252, 256, 256, 3)
Number of round images:  (1360, 256, 256, 3)


In [39]:
img_set = np.append(lever_imgs,round_imgs,axis=0)
# label_zeros = np.zeros((len(lever_imgs),1), np.float32)
# label_ones = np.ones((len(round_imgs),1), np.float32)
label_zeros = [[0.]]*len(lever_imgs)
label_ones = [[1.]]*len(round_imgs)

label_zeros.extend(label_ones)
label = label_zeros 

## Data split into train and test set
x_train, x_test, y_train, y_test = train_test_split(np.array(img_set,dtype=np.float32),np.array(label,dtype=np.float32),test_size=0.2, random_state=1)

In [40]:
y_train.shape

(2889, 1)

In [41]:
print('Number of train sets: ',x_train.shape[0])
print('Number of test sets: ',x_test.shape[0])

Number of train sets:  2889
Number of test sets:  723


### Hyperparameters

In [42]:
lr = 0.001
learning_epoch = 100
batch_size = 32

In [43]:
tf.reset_default_graph()

### CNN Model build
- Convolution -> Pooling -> Convolution -> Pooling -> FC

In [44]:
X = tf.placeholder(shape = [None, 256, 256, 3], dtype=tf.float32, name='X')
Y = tf.placeholder(tf.uint8, shape = [None,1], name='Y')
Y_ohe = tf.one_hot(Y, 2)
Y_ohe1 = tf.reshape(Y_ohe, [-1, 2])


conv1 = tf.layers.conv2d(inputs=X, kernel_size=[5,5], strides=(1,1),filters=8, padding='same', name='conv1', activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(conv1,pool_size=[4,4],strides=[4,4],name='pool1')

conv2 = tf.layers.conv2d(inputs=pool1, kernel_size=[5,5], strides=[1,1],filters=16, padding='SAME', name='conv2', activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(conv2,pool_size=[4,4],strides=[4,4],name='pool2')

flat = tf.layers.flatten(pool2, name='flat')

dense1=tf.layers.dense(inputs=flat, units=2000, activation=tf.nn.relu, name='dense1', kernel_initializer = tf.contrib.layers.xavier_initializer())
dense2=tf.layers.dense(inputs=dense1, units=800, activation=tf.nn.relu, name='dense2', kernel_initializer = tf.contrib.layers.xavier_initializer())
logit = tf.layers.dense(inputs=dense2, units=2, name='logits')


## Estimation result
est_res = tf.argmax(tf.nn.softmax(logit),1)

## Cost function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit,labels=Y_ohe1))

## Optimization of CNN model
optimizer = tf.train.AdamOptimizer(learning_rate = lr).minimize(cost)


correct_prediction = tf.equal(est_res,tf.argmax(Y_ohe1,1))

## Calculate accuracy
accuracy = tf.reduce_sum(tf.cast(correct_prediction,tf.float32))

In [45]:
def train(x_tr, y_tr):
    return sess.run([cost, optimizer, accuracy], feed_dict = {X: x_tr, Y: y_tr})

def test_est_acc(x_test, y_test):
    return sess.run([est_res, accuracy,tf.cast(correct_prediction,tf.float32)], feed_dict = {X: x_test, Y: y_test})

In [46]:
## Tensorflow session 
sess = tf.Session()
sess.run(tf.global_variables_initializer())

##
tr_batch_len = x_train.shape[0]//batch_size+1

for epoch in range(learning_epoch):
    data_index = list(np.arange(0,x_train.shape[0]))
    random.shuffle(data_index)
    co = 0
    acc = 0

    ## Training
    for batch in range(tr_batch_len):
        ## Train model by mini-batch dataset
        y_tr_input = y_train[data_index[batch*batch_size:(batch+1)*batch_size]]
        x_tr_input = x_train[data_index[batch*batch_size:(batch+1)*batch_size],:,:,:]
        c, _, a = train(x_tr_input, y_tr_input)
        
        ## Accuracy 
        co+=c
        acc += a

    co /= tr_batch_len
    acc /= x_train.shape[0]
    
    if epoch%10==0:
        print('Train -------------')
        print('Epoch: ',epoch,' / ',learning_epoch)
        print('Cost: ',co)
        print('Accuracy: ',acc,'\n\n')
    
        ## Test
        test_batch_len = x_test.shape[0]//batch_size+1
        acc_test = 0
        for batch_est in range(test_batch_len):
            x_test_input = x_test[batch_est*batch_size:(batch_est+1)*batch_size]
            y_test_input = y_test[batch_est*batch_size:(batch_est+1)*batch_size]
            er_test, ac_test,temp = test_est_acc(x_test_input,y_test_input)
            acc_test += ac_test
            
        acc_test /= x_test.shape[0]
        print('Test ---------------------')
        print(np.asarray(np.reshape(y_test_input,-1),dtype=np.int16),'\n',er_test)
        print('Accuracy: ',acc_test,'\n\n')

Train -------------
Epoch:  0  /  100
Cost:  0.6621739851249443
Accuracy:  0.6680512287988923 


Test ---------------------
[1 0 1 1 1 0 0 0 1 0 0 1 1 0 1 0 0 1 1] 
 [0 0 0 0 1 0 0 0 1 0 1 1 0 0 1 0 0 0 0]
Accuracy:  0.7634854771784232 


Train -------------
Epoch:  10  /  100
Cost:  0.08494673145341348
Accuracy:  0.9698857736240913 


Test ---------------------
[1 0 1 1 1 0 0 0 1 0 0 1 1 0 1 0 0 1 1] 
 [0 0 1 1 1 0 0 0 1 0 1 1 1 0 1 1 1 1 1]
Accuracy:  0.8672199170124482 


Train -------------
Epoch:  20  /  100
Cost:  0.005004231215914862
Accuracy:  0.9982692973347179 


Test ---------------------
[1 0 1 1 1 0 0 0 1 0 0 1 1 0 1 0 0 1 1] 
 [0 0 1 0 1 0 0 0 1 0 1 1 1 0 1 0 1 1 1]
Accuracy:  0.8976486860304288 


Train -------------
Epoch:  30  /  100
Cost:  0.006992282179483451
Accuracy:  0.9972308757355486 


Test ---------------------
[1 0 1 1 1 0 0 0 1 0 0 1 1 0 1 0 0 1 1] 
 [0 0 1 0 1 0 0 0 1 0 1 1 1 0 1 0 0 1 1]
Accuracy:  0.9073305670816044 


Train -------------
Epoch:  40  /  1